In [1]:
from bs4 import BeautifulSoup

In [2]:
import cchardet

In [3]:
import time

In [4]:
import re

In [5]:
import pandas as pd

In [6]:
#file2
#with open("2020_CORE/WR_2020_20210120185931_CORE_0002.xml", 'r') as f:
#     file = f.read()


In [7]:
#file3
with open("2020_CORE/WR_2020_20210120185931_CORE_0003.xml", 'r') as f:
     file = f.read()

In [8]:
t0=time.time()

In [9]:
soup = BeautifulSoup(file, 'xml')

In [10]:
t1=time.time()

In [11]:
time_taken=t1-t0
print(time_taken/60, "mins")

37.15894385178884 mins


In [12]:
#getting complete records
records = soup.findAll("REC")

In [13]:
len(records)

100000

In [14]:
uids = soup.find_all('UID')

In [15]:
UID=[]
for data in uids:
    UID.append(data.get_text())

In [16]:
titles = soup.find_all('title', type='source') #getting all journal etc. names

In [17]:
title=[]
for data in titles:
    title.append(data.get_text())

In [18]:
#getting all publishers by record
publish = soup.find_all('publishers')

In [19]:
publisher=[]
for i in range(len(publish)):
    publisher.append(publish[i].findChild('display_name').get_text()) # because it's a subtag do findChild

In [20]:
t2=time.time()

In [21]:
#convert records to string to get author names - takes around 13 minutes to run
allauthornames=[]
for i in range(len(records)):
    allauthornames.append(re.findall('<wos_standard>(.*?)</wos_standard>',records[i].decode()))

In [22]:
t3=time.time()

In [23]:
time_taken=t3-t2
print(time_taken/60, "mins")

13.41612833738327 mins


In [24]:
# function to get unique values
def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    #for x in unique_list:
     #   print(x)
    return unique_list

In [25]:
authors=[]
for i in range(len(allauthornames)):
    authors.append(unique(allauthornames[i]))

In [26]:
#getting author names and affiliations from the addresses tags
address = soup.find_all('addresses' )

In [27]:
#getting the number of authors in each record having affiliation
authors_w_aff=[]
for i in range(len(address)):
    authors_w_aff.append(len(unique(address[i].find_all('display_name'))))

In [28]:
add_names=[]
for data in address:
    add_names.append(data.get_text())

In [29]:
pattern = '\n\n\n\n\n\n' ### 6 \n -> new author/address combination in address

In [30]:
num_aff=[]
for i in range(len(add_names)):
    if(len(add_names[i]))>0:
        value=len(re.findall(pattern, add_names[i]))+1  #need to add 1 for records with data
    else: value = 0
    num_aff.append(value)

In [31]:
df=pd.DataFrame()

In [32]:
df['UID']=UID

In [33]:
df['Number of Affiliations'] = num_aff

In [34]:
df['No. of Authors with Aff'] = authors_w_aff

In [35]:
df['Source Title'] = title

In [36]:
df['Publisher'] = publisher

In [37]:
df['All Authors'] = authors

In [38]:
df['Authors with Aff text']=add_names

In [39]:
#df.to_csv("WOS2_description.csv", index=False)
df.to_csv("WOS3_description.csv", index=False)

In [40]:
#df.describe()#for file 2

In [41]:
df.head()

,UID,Number of Affiliations,No. of Authors with Aff,Source Title,Publisher,All Authors,Authors with Aff text
0,WOS:000526811600009,27,29,JOURNAL OF CLINICAL ONCOLOGY,AMER SOC CLINICAL ONCOLOGY,"[Delaloge, S, Piccart, M, Rutgers, E, Litiere,...","\n\n\nGustave Roussy, Villejuif, France\n\nGus..."
1,WOS:000560368303200,1,0,JOURNAL OF CLINICAL ONCOLOGY,AMER SOC CLINICAL ONCOLOGY,"[Scarpa, FJ, Paul, M, Wolfson, WA, Weiss, LM, ...","\n\n\nNeoGenom Labs, Aliso Viejo, CA USA\n\nNe..."
2,WOS:000547704900008,1,3,MOLECULAR SYSTEMS DESIGN & ENGINEERING,ROYAL SOC CHEMISTRY,"[Shi, JL, Sidky, H, Whitmer, JK]","\n\n\nUniv Notre Dame, Dept Chem & Biomol Engn..."
3,WOS:000564800900006,5,9,QUATERNARY SCIENCE REVIEWS,PERGAMON-ELSEVIER SCIENCE LTD,"[Toti, F, Bertini, A, Girone, A, Marino, M, Ma...","\n\n\nUniv Firenze, Dipartimento Sci Terra, Vi..."
4,WOS:000585840800019,3,4,PLANT PHYSIOLOGY,AMER SOC PLANT BIOLOGISTS,"[Belanger, S, Pokhrel, S, Czymmek, K, Meyers, BC]","\n\n\nDonald Danforth Plant Sci Ctr, St Louis,..."


In [42]:
df.tail()

,UID,Number of Affiliations,No. of Authors with Aff,Source Title,Publisher,All Authors,Authors with Aff text
99995,WOS:000516426900002,1,2,CLUSTER COMPUTING-THE JOURNAL OF NETWORKS SOFT...,SPRINGER,"[Mishra, SK, Manjula, R]","\n\n\nVIT Univ, Sch Comp Sci & Engn SCOPE, Vel..."
99996,WOS:000558533200095,1,6,ENERGY,PERGAMON-ELSEVIER SCIENCE LTD,"[Diaz, M, Epelde, E, Tabernilla, Z, Ateka, A, ...","\n\n\nUniv Basque Country, Dept Chem Engn, UPV..."
99997,WOS:000563265100001,3,6,SMALL,WILEY-V C H VERLAG GMBH,"[Zaker, Y, Ashenfelter, BA, Bhattarai, B, Diem...","\n\n\nUniv Toledo, Dept Chem & Biochem, 2801 W..."
99998,WOS:000529877600078,6,10,BIOMOLECULES,MDPI,"[Szwed, A, Milowska, K, Michlewska, S, Moreno,...","\n\n\nUniv Lodz, Fac Biol & Environm Protect, ..."
99999,WOS:000509617400030,3,6,FISH & SHELLFISH IMMUNOLOGY,ACADEMIC PRESS LTD- ELSEVIER SCIENCE LTD,"[Sirisena, DMKP, Tharuka, MDN, Liyanage, DS, J...","\n\n\nJeju Natl Univ, Dept Marine Life Sci, Ar..."
